# Introduction, Shortcomings, and Goals

The following collab is used to create a more dynamic chord progression model. In the given state, the LSTM model is overfitting. This is most likely due to the way in which the dataset was formed. The problem is multiclassification, which means the dataset needs to mimic this. As of right now, multiple inputs with different outputs are given which confuses the model.  

The final goal is to use this LSTM model as a discriminator for a GAN network.

# Imports

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import collections
from sklearn.preprocessing import MinMaxScaler

print(torch.__version__)


import os
import shutil
import copy

!pip install mido
import mido

1.8.1+cu101
     |████████████████████████████████| 61kB 7.6MB/s 


# Model Creation

In [ ]:
class SimpleLSTM(nn.Module):
  def __init__(self,
               batch_size,
               hidden_dim,
               lstmLayers):
    #class input saves
    super(SimpleLSTM,self).__init__()
    #initialize weights with normal distribution
    x = nn.Linear(100, 100)
    nn.init.normal_(x.weight, mean=0, std=1.0)

    #class variables
    self.batch_size = batch_size
    self.hidden_dim = hidden_dim
    self.lstmLayers = lstmLayers

    #Embed each chord for better training
    #self.chord_embeddings = nn.Embedding(lstmLayers,lstmLayers)

    #LSTM network setup
    self.lstm = nn.LSTM(hidden_dim,hidden_dim,num_layers=self.lstmLayers)
    #initialize hidden states
    self.hidden = self.init_hidden()

  def init_hidden(self):
    #set hidden initial states per chord progression
    return (autograd.Variable(torch.zeros(self.lstmLayers, self.batch_size, self.hidden_dim)),
            autograd.Variable(torch.zeros(self.lstmLayers, self.batch_size, self.hidden_dim)))
  
  def forward(self,chord):
    #embeds = self.chord_embeddings(chord_progression)

    #input lstm(<chords in sequence>,<sequence/batch_size>, <features/notes>)

    #chord_progression.view(self.lstmLayers,1,self.hidden_dim)
    lstm_out, self.hidden = self.lstm(chord.view(1,self.batch_size,self.hidden_dim),
                                      self.hidden)
    
    return lstm_out, self.hidden

# Load Dataset

In [ ]:
tensorLocation = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Datasets/AllDataSetsCombined/Essential_Midi_ProgressionsOnly/tensor_essentialDataset.pt"
dataset = torch.load(tensorLocation)

print(dataset.shape)

(1416, 4, 3)


In [ ]:
#convert dataset to data_loader. Note: torch.Tensor is different than 'convert_to_tensor' when creating dataset
max_number = 108

#set datatype to long int
dataConverted = torch.tensor(dataset.numpy(),dtype=torch.float32)
#normalize via midi protocol max
dataConverted = torch.div(dataConverted,max_number)

#####################EVAL & TRAIN SEPERATION###########################
eval_percentage = .10 #10%
total_files = len(dataConverted)
eval_number = round(total_files * eval_percentage) # number of files to keep to nearest integer
print("eval percentage: {}\ntotal files in dataset: {}\nExpected number of files in eval dataset: {}\n".format(
    eval_percentage,total_files,eval_number))

data_train = dataConverted[0:total_files-eval_number]
data_eval = dataConverted[total_files-eval_number:]
print("Eval Shape: {}\nTraining Shape: {}\nDatasetLength == EvalShape+TrainShape: {}".format(
    data_eval.shape,data_train.shape,(data_eval.shape[0]+data_train.shape[0])==dataConverted.shape[0]))



assert data_train.shape[0] == total_files-eval_number, "Training dataset does not match seperation of evaluation percentage\n \
Training Dataset Shape ({})\n Expected first dimension: {}\n".format(data_train.shape,total_files-eval_number)
#train & eval loaders#
loader_train = torch.utils.data.DataLoader(data_train,batch_size=32,shuffle=True)
loader_eval = torch.utils.data.DataLoader(data_eval,batch_size=32,shuffle=False)


print()
print(loader_train.dataset.type())
print(loader_train.dataset.shape)
print(loader_eval.dataset.type())
print(loader_eval.dataset.shape)

eval percentage: 0.1
total files in dataset: 1416
Expected number of files in eval dataset: 142

Eval Shape: torch.Size([142, 4, 3])
Training Shape: torch.Size([1274, 4, 3])
DatasetLength == EvalShape+TrainShape: True

torch.FloatTensor
torch.Size([1274, 4, 3])
torch.FloatTensor
torch.Size([142, 4, 3])


In [ ]:
#see that data loop is in form we want.
#the same loop is used for training
for chords in loader_train.dataset:
  chordLength = len(chords) - 1
  print("Start of Sequence")
  for i, chord in enumerate(chords):
    if i != (chordLength):
      print(chord)
    else:
      print(chord)
      print(chords[0],"\n-------------------------------")
  #chord progression size
  print(chord.shape,"\n")

Streaming output truncated to the last 5000 lines.
tensor([0.5185, 0.5463, 0.5833])
tensor([0.5648, 0.6019, 0.6296]) 
-------------------------------
torch.Size([3]) 

Start of Sequence
tensor([0.5000, 0.5370, 0.5648])
tensor([0.5185, 0.5463, 0.5833])
tensor([0.5370, 0.5648, 0.6019])
tensor([0.5648, 0.6019, 0.6296])
tensor([0.5000, 0.5370, 0.5648]) 
-------------------------------
torch.Size([3]) 

Start of Sequence
tensor([0.5648, 0.6019, 0.6296])
tensor([0.5463, 0.5833, 0.6111])
tensor([0.5000, 0.5370, 0.5648])
tensor([0.5648, 0.6019, 0.6296])
tensor([0.5648, 0.6019, 0.6296]) 
-------------------------------
torch.Size([3]) 

Start of Sequence
tensor([0.5833, 0.6111, 0.6481])
tensor([0.5000, 0.5370, 0.5648])
tensor([0.5648, 0.6019, 0.6296])
tensor([0.5463, 0.5833, 0.6111])
tensor([0.5833, 0.6111, 0.6481]) 
-------------------------------
torch.Size([3]) 

Start of Sequence
tensor([0.5000, 0.5370, 0.5648])
tensor([0.5000, 0.5370, 0.5648])
tensor([0.5833, 0.6111, 0.6481])
tensor([0.564

# Initialization

In [ ]:
#Check for cuda ---> not needed if on google collab
if torch.cuda.is_available():
  device = torch.device('cuda')
  print("Device Name: {}".format(device))
else:
  print("no device found")

Device Name: cuda


In [ ]:
#input variables
batch_size = 32
hidden_dim = 3 #number of notes per chord
lstm_layers = 4 #place_holder based on how model is trained.
  #However, dataset is organized by chord progression where each progression has 4 chords. (12 notes total)

#initalize model, optimizer, and loss function
loss_function = nn.MSELoss()

model = SimpleLSTM(batch_size=batch_size,
                   hidden_dim=hidden_dim,
                   lstmLayers=lstm_layers)
model.to(device)
optimizer = optim.SGD(model.parameters(),lr=.1)

print(model.parameters)

<bound method Module.parameters of SimpleLSTM(
  (lstm): LSTM(3, 3, num_layers=4)
)>


In [ ]:
#model info
print("Model Information")
print(model)

#dataset info
print("\nDataset Features")
print(loader_train.dataset.type())

Model Information
SimpleLSTM(
  (lstm): LSTM(3, 3, num_layers=4)
)

Dataset Features
torch.FloatTensor


# IGNORE

In [ ]:
def evaluate_epoch(loader_eval,batch_size,hidden_dim,model,epoch=0,loss=0):
  """
epoch and loss set to zero if you want to use this just for accuracy calculations, else feed those in, if using with training.
  """
  #eval_testing

  eval_load = iter(loader_eval)


  # evaluate model:
  model.eval()
  with torch.no_grad():
    total = 0
    correct = 0
    sum_correct = 0
    for data in eval_load:
      chords = data.cuda()
      local_batch = chords.shape[0]
      chords = chords.resize(4,local_batch,hidden_dim)

      chordLength = len(chords) - 1
      for i, chord_list in enumerate(chords):
        if chord_list.shape[0] != 32:
          continue

        model.hidden = model.init_hidden()

        last_chord, _allChords = model(chord_list)

        
        current_chord = torch.mul(chords[i],max_number)
        current_chord = torch.round(current_chord).view(batch_size,hidden_dim)

        eval_chords = torch.mul(last_chord,max_number)
        eval_chords = torch.round(eval_chords).view(batch_size,hidden_dim)

        if i != chordLength:
          target_chords = torch.mul(chords[i+1],max_number)
        else:
          target_chords = torch.mul(chords[0],max_number)

        target_chords = torch.round(target_chords)

        for i, chord in enumerate(eval_chords):
          #print(i)
          #print("input chord: {}".format(current_chord[i]))
          #print("model chord: {}".format(eval_chords[i]))
          #print("expected_chords: {}\n".format(target_chords[i]))
          total += 1
          if torch.equal(chord,target_chords[i]):
            correct += 1
            # print("input chord: {}".format(current_chord[i]))
            # print("model chord: {}".format(eval_chords[i]))
            # print("expected_chords: {}\n".format(target_chords[i]))
          if torch.sum(target_chords[i]) == torch.sum(chord):
            sum_correct += 1

    accuracy = (correct/total) * 100
    print("For Epoch: {}\nLoss: {}\nAccuracy: {}%\n".format(
          epoch,loss,accuracy))

evaluate_epoch(loader_eval,batch_size,hidden_dim,model)

/usr/local/lib/python3.7/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


RuntimeError: ignored

In [ ]:
# training
torch.autograd.set_detect_anomaly(True)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
#model.train()
i = 0

#initialize loader for batches
for epoch in range(15):
  model.train()
  loader = iter(loader_train)
  itemLoss = 0
  print("-------------------------------------------------------------------")
  for data in loader:
    #print(data.shape)
    chords = data.cuda()
    local_batch = chords.shape[0]
    chords = chords.resize(4,local_batch,hidden_dim)

    chordLength = len(chords) - 1
    for i, chord_list in enumerate(chords):
      if chord_list.shape[0] != 32:
        continue
      
      #STEP 1: Reset gradient/chord progression
      model.zero_grad()
      #STEP 2: Reset hidden_states
      model.hidden = model.init_hidden()
      # STEP 3. Run our forward pass.
      chords_list = autograd.Variable(chord_list)
      chord_list = chord_list
      
      last_chord, _allChords = model(chord_list)

      # STEP 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
        #if statement is to allow for looping when ending a chord-progression
      last_chord = last_chord.view(batch_size,hidden_dim)
      if i != chordLength:
        labels = chords[i+1].resize(batch_size,hidden_dim)
        loss = loss_function(last_chord, labels)
        loss.backward()
      else:
        #loss function connects last chord to first chord, "repeat a progression"
        labels = chords[0].resize(batch_size,hidden_dim)
        loss = loss_function(last_chord, labels)
        loss.backward()
      itemLoss += loss.item()
      #print(itemLoss)
      #STEP 5: Update weights before next set
      optimizer.step()

  evaluate_epoch(loader_eval,batch_size,hidden_dim,model,epoch=epoch,loss=itemLoss)

  #use below if evaluation function is unused
  #print("For Epoch {}: Loss: {}".format(epoch,itemLoss))

-------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


For Epoch: 0
Loss: 15.439056660979986
Accuracy: 0.0%

-------------------------------------------------------------------
For Epoch: 1
Loss: 4.174294181168079
Accuracy: 0.0%

-------------------------------------------------------------------
For Epoch: 2
Loss: 1.7157559865154326
Accuracy: 0.0%

-------------------------------------------------------------------


KeyboardInterrupt: ignored

#Working training (complete steps)

REVIEW: Last time was used the abcd idea, to make sure the output is always a shifted copy of the input. This way, the "next" note is always what we want next. Refer to scratch notes in desk for reminder.

Dataset trains, but isn't learning to well. After testing, I have found that model outputs ~1 note off for any given position. some have a 3 notes off by 1. Other have only 1 or 2 notes that off, and the others are correct. This can be seen with single note accuracy.

NEED TO DO:  
-Create noteToMidi() for audio testing purposes  
-create evaluation/inferencing class for model  
This needs to be done so we can evaluate the model with 1,2,3,4 input time sequences.



In [ ]:
class CompleteLSTM(nn.Module):
  def __init__(self,
               batch_size,
               hidden_dim,
               lstmLayers,
               input_size):
    #class input saves
    super(CompleteLSTM,self).__init__()
    #initialize weights with normal distribution
    x = nn.Linear(100, 100)
    nn.init.normal_(x.weight, mean=0, std=1.0)

    #class variables
    self.batch_size = batch_size
    self.hidden_dim = hidden_dim
    self.lstmLayers = lstmLayers
    self.input_size = input_size

    #Embed each chord for better training
    #self.chord_embeddings = nn.Embedding(lstmLayers,lstmLayers)

    #LSTM network setup
    self.lstm = nn.LSTM(input_size,hidden_dim,num_layers=self.lstmLayers)
    #initialize hidden states
    self.hidden = self.init_hidden()

  def init_hidden(self):
    #set hidden initial states per chord progression
    return (autograd.Variable(torch.zeros(self.lstmLayers, self.batch_size, self.hidden_dim)),
            autograd.Variable(torch.zeros(self.lstmLayers, self.batch_size, self.hidden_dim)))
  
  def forward(self,input):
    #input lstm(<time>,<batch>, <features>)

    #chord_progression.view(self.lstmLayers,1,self.hidden_dim)
    lstm_out, self.hidden = self.lstm(input, self.hidden)
    
    return lstm_out, self.hidden

In [ ]:
#train & eval loaders#
loader_train = torch.utils.data.DataLoader(data_train,batch_size=1,shuffle=True)
loader_eval = torch.utils.data.DataLoader(data_eval,batch_size=1,shuffle=False)

In [ ]:
#input variables
batch_size = 1 #batch_size
hidden_dim = 3 #important features
lstm_layers = 4 #number of LSTM stacked layers
input_size = 3 #NUMBER OF FEATURES?
time_seq = 4 #number of instances in time
  #However, dataset is organized by chord progression where each progression has 4 chords. (12 notes total)

#initalize model, optimizer, and loss function
loss_function = nn.MSELoss()

model = CompleteLSTM(batch_size=batch_size,
                   hidden_dim=hidden_dim,
                   lstmLayers=lstm_layers,
                   input_size=input_size)
model.to(device)
optimizer = optim.Adam(model.parameters(),lr=.001)

print(model.parameters)

<bound method Module.parameters of CompleteLSTM(
  (lstm): LSTM(3, 3, num_layers=4)
)>


In [ ]:
def floatToMidi(tensorArray,maxNumber = 108):
  output = torch.mul(tensorArray,maxNumber)
  output = torch.round(output)

  return output


def firstRowToLostRow(tensorArray):
  """
  return:
    original tensor but first row is moved to the end
  """
  tensorArray = tensorArray.view(4,3)
  clone = tensorArray.clone()
  firstRow = clone[0]

  firstRow = firstRow.view(1,firstRow.shape[0])

  other_array = clone[1:]

  finalTensor = torch.cat((other_array,firstRow))
  return finalTensor

test = loader_train.dataset[0].clone()
print(test)

newTensor = firstRowToLostRow(test)
print()
print(newTensor)

tensor([[0.5093, 0.5463, 0.5741],
        [0.5556, 0.5926, 0.6204],
        [0.5741, 0.6111, 0.6389],
        [0.5926, 0.6204, 0.6574]], device='cpu')

tensor([[0.5556, 0.5926, 0.6204],
        [0.5741, 0.6111, 0.6389],
        [0.5926, 0.6204, 0.6574],
        [0.5093, 0.5463, 0.5741]], device='cpu')


In [ ]:
# training
torch.autograd.set_detect_anomaly(True)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
#model.train()
model.cuda()
#initialize loader for batches
for epoch in range(1):
  model.train()
  loader = iter(loader_train)
  itemLoss = 0
  print("-------------------------------------------------------------------")
  for i,data in enumerate(loader):
    #print(data.shape)
    data = data.cuda()
    chords = autograd.Variable(data)

    local_batch = chords.shape[0]
    chords = chords.view(time_seq,local_batch,input_size)

    #STEP 1: Reset gradient
    model.zero_grad()
    #STEP 2: Reset hidden_states
    model.hidden = model.init_hidden()
    # STEP 3. Run our forward pass.
    last_chord, _allChords = model.forward(chords)

    #print(last_chord.shape)
    # STEP 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
      #if statement is to allow for looping when ending a chord-progression
    
    labels = data.view(4,3)
    labels = firstRowToLostRow(labels)

    modelOutput = last_chord.view(4,3)

    loss = loss_function(modelOutput[-1], labels[-1])
    loss.backward()

    itemLoss += loss.item()
    #print(itemLoss)
    #STEP 5: Update weights before next set
    optimizer.step()

  model_requirements = evaluate_epoch_large(loader_eval,batch_size,hidden_dim,model,epoch=epoch,loss=itemLoss)
  if model_requirements == True:
    break

  #use below if evaluation function is unused
  #print("For Epoch {}: Loss: {}".format(epoch,itemLoss))

-------------------------------------------------------------------


NameError: ignored

In [ ]:
#save model weights - MAKE SURE TO NOT OVERWRITE
model_weights_save_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/state_dict_model_example.pt"
torch.save(model.state_dict(),model_weights_save_path)

In [ ]:
#LOAD MODEL WEIGHTS
load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/state_dict_model_4input.pt"
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

---  
model evaluator  

In [ ]:
def evaluate_epoch_large(loader_eval,batch_size,hidden_dim,model,epoch=0,loss=0,testRun = False,perfectThreshold=100):
  """
epoch and loss set to zero if you want to use this just for accuracy calculations, else feed those in, if using with training.
  """
  #eval_testing

  eval_load = iter(loader_eval)


  # evaluate model:
  model.eval()
  with torch.no_grad():
    total = 0
    correct =  0
    single_note_total = 0
    single_note_correct = 0
    for j, data in enumerate(loader_eval):
      data = data.cuda()
      chords = data

      local_batch = chords.shape[0]
      chords = chords.resize(4,local_batch,hidden_dim)
      #reset hidden states
      model.hidden = model.init_hidden()

      #run model
      last_chord, _allChords = model(chords)

      # print("START")
      # print(last_chord)
      # print(chords)
      # print("END")
      ###################################################Collect model output and expected label and convert back to midi integers################################
      last_chords = floatToMidi(last_chord).view(4,3) #STEP 1
      ###add 1 to model for accuracy buff###
      # last_chords = torch.add(last_chords,1)
      
      labels = chords.view(4,3) #STEP 2
      labels = firstRowToLostRow(labels)
      target_chords = floatToMidi(labels).view(4,3)

      assert labels.shape == target_chords.shape, "labels shape {} != target_chords shape {}".format(labels.shape,target_chords.shape)
      tmp = floatToMidi(_allChords[0])
      if testRun == True:
        print("-----------------------------test data--------------------------")
        print("expected chords:\n{}".format(target_chords))
        #print("hidden state chords:\n{}".format(tmp))
        print("model output:\n{}".format(last_chords))
        print("----------------------------------------------------------------")


      if torch.equal(last_chords[-1],target_chords[-1]):
        correct += 1
      
      total += 1

      for idx, note in enumerate(labels[-1]):
        if torch.equal(last_chords[-1][idx],target_chords[-1][idx]):
          single_note_correct += 1
        else:
          pass
          #print(last_chords,target_chords)

    accuracy = (correct/total) * 100

    single_note_total = 3 * total
    single_note_accuracy = single_note_correct/single_note_total * 100

    print("For Epoch: {}\nLoss: {}\nAccuracy: {:.2f}%\nTotal Tests: {}\nSingle Note Accuracy: {:.2f}%".format(
          epoch,loss,accuracy,total,single_note_accuracy))
    
    #threshold default at 100%
    perfect = False
    if accuracy >= perfectThreshold:
      perfect = True

    return perfect

evaluate_epoch_large(loader_eval,batch_size,hidden_dim,model,testRun = True) 

/usr/local/lib/python3.7/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


-----------------------------test data--------------------------
expected chords:
tensor([[62., 66., 69.],
        [60., 64., 67.],
        [59., 62., 66.],
        [52., 55., 59.]])
model output:
tensor([[27., 25., 26.],
        [40., 42., 42.],
        [46., 49., 52.],
        [52., 55., 59.]])
----------------------------------------------------------------
-----------------------------test data--------------------------
expected chords:
tensor([[50., 54., 57.],
        [55., 59., 62.],
        [59., 62., 66.],
        [52., 55., 59.]])
model output:
tensor([[27., 25., 26.],
        [40., 42., 42.],
        [46., 49., 51.],
        [52., 55., 59.]])
----------------------------------------------------------------
-----------------------------test data--------------------------
expected chords:
tensor([[60., 64., 67.],
        [59., 62., 66.],
        [59., 62., 66.],
        [52., 55., 59.]])
model output:
tensor([[27., 25., 26.],
        [40., 42., 42.],
        [46., 49., 52.],
  

True

In [ ]:
#MODEL TOOL FUNCTIONS (Inference)
def _singleChordToMidi(inputChord,velocity = 64,program=12,chordLength=1028):
  inputChord = inputChord.view(3).tolist()

  track = mido.MidiTrack()

  #track.append(mido.Message('program_change',program=program,time=0))
  for idx, note in enumerate(inputChord):
    if idx != 0:
      track.append(mido.Message('note_on',note=note,velocity=velocity,time=0))
    else:
      track.append(mido.Message('note_on',note=note,velocity=velocity,time=chordLength))
  #turn off first chords & turn on appended chord
  for idx, note in enumerate(inputChord):
    if idx != 0:
      track.append(mido.Message('note_off',note=note,velocity=velocity,time=0))
    else:
      track.append(mido.Message('note_off',note=note,velocity=velocity,time=chordLength))
  return track

def ChordConverter(inputChords,outputChords,chordLength=3):

  num_of_chords = len(inputChords)
  if len(inputChords) == 4:
    inputChords = firstRowToLostRow(inputChords)
    inputChords.view(num_of_chords,3)

  mid = mido.MidiFile()
  track = mido.MidiTrack()
  for idx, chord in enumerate(inputChords):
    track += _singleChordToMidi(inputChords[idx].view(1,chordLength))
    
  for idx, chord in enumerate(outputChords):
    track += _singleChordToMidi(inputChords[idx].view(1,chordLength))
  
  mid.tracks.append(track)
  return mid

def _modelInferencePrint(input, output):
  print("Model Inputs & Outputs")
  print("Test Beginning\nmodelInput:{}\nmodelOutput:{}\n".format(input,output))

def _floatToMidi(tensorArray,maxNumber = 108):
  output = torch.mul(tensorArray,maxNumber)
  output = torch.round(output)
  return output

def modelInference(inputChord,model):
  input = inputChord.view(len(inputChord),1,3)
  model.eval()
  with torch.no_grad():
    model.hidden = model.init_hidden()
    output, _ = model(input)
  #convert back to midi note values
  #print(output)
  first_chord = _floatToMidi(input)
  next_chord = _floatToMidi(output)
  return first_chord, next_chord


def floatToInt(inputTensor):
  output = torch.tensor(inputTensor,dtype=torch.int64)
  return output

In [ ]:
#for idx, chords in enumerate(loader_eval):
#  print(chords)

inputChord = loader_eval.dataset[0].cuda()

#model inference
input_chord, output_chord = modelInference(inputChord,model)
#print information about inference
_modelInferencePrint(input_chord,output_chord)


#convert to integer
input = floatToInt(input_chord)
output = floatToInt(output_chord)

mid = ChordConverter(input,output[-1])

for mess in mid.tracks[0]:
  print(mess)

mid.save('test_track.mid')

Model Inputs & Outputs
Test Beginning
modelInput:tensor([[[52., 55., 59.]],

        [[62., 66., 69.]],

        [[60., 64., 67.]],

        [[59., 62., 66.]]])
modelOutput:tensor([[[27., 25., 26.]],

        [[40., 42., 42.]],

        [[46., 49., 52.]],

        [[52., 55., 59.]]])

note_on channel=0 note=62 velocity=64 time=1028
note_on channel=0 note=66 velocity=64 time=0
note_on channel=0 note=69 velocity=64 time=0
note_off channel=0 note=62 velocity=64 time=1028
note_off channel=0 note=66 velocity=64 time=0
note_off channel=0 note=69 velocity=64 time=0
note_on channel=0 note=60 velocity=64 time=1028
note_on channel=0 note=64 velocity=64 time=0
note_on channel=0 note=67 velocity=64 time=0
note_off channel=0 note=60 velocity=64 time=1028
note_off channel=0 note=64 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=0
note_on channel=0 note=59 velocity=64 time=1028
note_on channel=0 note=62 velocity=64 time=0
note_on channel=0 note=66 velocity=64 time=0
note_off channel=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


# Test Models

In [ ]:
#input variables
batch_size = 1 #batch_size
hidden_dim = 3 #important features
lstm_layers = 8 #number of LSTM stacked layers
input_size = 3 #NUMBER OF FEATURES?
time_seq = 4 #number of instances in time
  #However, dataset is organized by chord progression where each progression has 4 chords. (12 notes total)

#initalize model, optimizer, and loss function
loss_function = nn.MSELoss()

test_model = CompleteLSTM(batch_size=batch_size,
                   hidden_dim=hidden_dim,
                   lstmLayers=lstm_layers,
                   input_size=input_size)
test_model.to(device)
optimizer = optim.Adam(test_model.parameters(),lr=.001)

print(test_model.parameters)

<bound method Module.parameters of CompleteLSTM(
  (lstm): LSTM(3, 3, num_layers=8)
)>


In [ ]:
optimizer = optim.Adam(test_model.parameters(),lr=.001)

In [ ]:
# training
torch.autograd.set_detect_anomaly(True)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

test_model.cuda()

seqInput = 1
#initialize loader for batches
for epoch in range(500):
  test_model.train()
  loader = iter(loader_train)
  itemLoss = 0
  print("-------------------------------------------------------------------")
  for i,data in enumerate(loader):
    #print(data.shape)
    data = data.cuda()
    chords = autograd.Variable(data)

    local_batch = chords.shape[0]
    chords = chords.view(time_seq,local_batch,input_size)

    #STEP 1: Reset gradient
    test_model.zero_grad()
    #STEP 2: Reset hidden_states
    test_model.hidden = test_model.init_hidden()
    # STEP 3. Run our forward pass.
    last_chord, _allChords = test_model.forward(chords[:seqInput])

    #print(last_chord.shape)
    # STEP 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
      #if statement is to allow for looping when ending a chord-progression
    
    labels = data.view(4,3)

    modelOutput = last_chord.view(seqInput,3)

    loss = loss_function(modelOutput[-1], labels[seqInput]) #seq input is the "next input for loss calculations, model output is always last position
    loss.backward()

    itemLoss += loss.item()
    #print(itemLoss)
    #STEP 5: Update weights before next set
    optimizer.step()

  model_requirements = evaluate_epoch_test(loader_eval,batch_size,hidden_dim,test_model,epoch=epoch,loss=itemLoss)
  if model_requirements == True:
    break

  #use below if evaluation function is unused
  #print("For Epoch {}: Loss: {}".format(epoch,itemLoss))

-------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


For Epoch: 0
Loss: 79.78494071757223
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
For Epoch: 1
Loss: 2.8271980332926887
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
For Epoch: 2
Loss: 2.840230995100228
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
For Epoch: 3
Loss: 2.846274095415197
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
For Epoch: 4
Loss: 2.834158894553184
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
For Epoch: 5
Loss: 2.8415992740681304
Accuracy: 0.00%
Total Tests: 142
Single Note Accuracy: 0.00%
-------------------------------------------------------------------
Fo

KeyboardInterrupt: ignored

In [ ]:
def evaluate_epoch_test(loader_eval,batch_size,hidden_dim,model,epoch=0,loss=0,testRun = False,perfectThreshold=100):
  """
epoch and loss set to zero if you want to use this just for accuracy calculations, else feed those in, if using with training.
  """
  #eval_testing

  eval_load = iter(loader_eval)


  # evaluate model:
  model.eval()
  with torch.no_grad():
    total = 0
    correct =  0
    single_note_total = 0
    single_note_correct = 0

    seqInput = 3
    for j, data in enumerate(eval_load):
      data = data.cuda()
      chords = data

      local_batch = chords.shape[0]
      chords = chords.resize(4,local_batch,hidden_dim)
      #reset hidden states
      model.hidden = model.init_hidden()
      #run model
      last_chord, _allChords = model(chords[:seqInput])

      ###################################################Collect model output and expected label and convert back to midi integers################################
      last_chords = floatToMidi(last_chord).view(seqInput,3) #STEP 1
      ###add 1 to model for accuracy buff###
      # last_chords = torch.add(last_chords,1)
      
      labels = data.view(4,3) #STEP 2
      target_chords = floatToMidi(labels).view(4,3)

      assert labels.shape == target_chords.shape, "labels shape {} != target_chords shape {}".format(labels.shape,target_chords.shape)
      tmp = floatToMidi(_allChords[0])
      if testRun == True:
        print("-----------------------------test data--------------------------")
        print("expected chords:\n{}".format(target_chords))
        print("hidden state chords:\n{}".format(tmp))
        print("model output:\n{}".format(last_chords))
        print("----------------------------------------------------------------")


      if torch.equal(last_chords[-1],target_chords[seqInput]):
        correct += 1
      
      total += 1

      for idx, note in enumerate(labels[-1]):
        if torch.equal(last_chords[-1][idx],target_chords[-1][idx]):
          single_note_correct += 1
        else:
          pass
          #print(last_chords,target_chords)

    accuracy = (correct/total) * 100

    single_note_total = 3 * total
    single_note_accuracy = single_note_correct/single_note_total * 100

    print("For Epoch: {}\nLoss: {}\nAccuracy: {:.2f}%\nTotal Tests: {}\nSingle Note Accuracy: {:.2f}%".format(
          epoch,loss,accuracy,total,single_note_accuracy))
    
    #threshold default at 100%
    perfect = False
    if accuracy >= perfectThreshold:
      perfect = True

    return perfect

evaluate_epoch_test(loader_eval,batch_size,hidden_dim,test_model,testRun = True) 

/usr/local/lib/python3.7/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


-----------------------------test data--------------------------
expected chords:
tensor([[52., 55., 59.],
        [62., 66., 69.],
        [60., 64., 67.],
        [59., 62., 66.]])
hidden state chords:
tensor([[[-72.,  79.,  86.]],

        [[ 90., -92.,  97.]],

        [[103., -99., -90.]],

        [[ 85.,  88.,  77.]]])
model output:
tensor([[57., 61., 64.],
        [78., 82., 74.],
        [85., 88., 77.]])
----------------------------------------------------------------
-----------------------------test data--------------------------
expected chords:
tensor([[52., 55., 59.],
        [50., 54., 57.],
        [55., 59., 62.],
        [59., 62., 66.]])
hidden state chords:
tensor([[[-70.,  77.,  85.]],

        [[ 90., -92.,  96.]],

        [[103., -99., -90.]],

        [[ 85.,  88.,  77.]]])
model output:
tensor([[57., 61., 64.],
        [78., 82., 74.],
        [85., 88., 77.]])
----------------------------------------------------------------
-----------------------------test 

False